# 회원가입 및 로그인 프로그램

#### 따로 DB를 구축하지 않아 회원가입 정보는 해당 파이썬 파일이 위치한 디렉토리에 csv파일 형태로 저장됨.
#### 주민번호는 3DES 암호화 모듈을 통해 뒤 6자리만 암호화하여 저장하도록 만들었다.
#### 비밀번호는 해시 알고리즘 SHA256을 사용해 암호화하여 저장하도록 만들었다.

### 모듈 import

pycryptodome 모듈외 다른 모듈 설치 필요.

In [ ]:
import crypto
import sys
sys.modules['Crypto'] = crypto
from crypto.Cipher import DES3
import hashlib
from hashlib import sha256
import os
import pandas as pd
import time

### class 선언

#### myDES : 가입자가 입력한 정보 중 주민번호 뒤 6자리와 비밀번호를 각각 암호화하여 가입정보를 저장.
#### Login : read_file() : 가입자 정보가 담긴 파일 가져오기. / login_pw_enc() : 로그인시 입력한 비밀번호 암호화하기. 
####         / check_info() : 로그인시 아이디와 비밀번호가 가입정보와 맞는지 체크하기 위해 read_file()에서 불러온 데이터에서 아이디와 패스워드만 추출.

In [ ]:
class myDES():
    def __init__(self, keytext, ivtext):
        hash = hashlib.sha256()
        hash.update(keytext.encode('utf-8'))
        key = hash.digest()
        self.key = key[:24]

        hash.update(ivtext.encode('utf-8'))
        iv = hash.digest()
        self.iv = iv[:8]

    def code_enc(self, code):
        cc = code[-7:-1]
        ccc = cc + '##'
        des3 = DES3.new(self.key, DES3.MODE_CBC, self.iv)
        encmsg = des3.encrypt(ccc.encode())
        enctxt = code[0:6] + '-' + str(encmsg)
        return enctxt

    def pw_enc(self, pw):
        sha = sha256()
        sha.update(pw.encode('utf-8'))
        enc_pw = sha.hexdigest()

        return enc_pw

    def save_file(self, name, enctxt, Id, enc_pw, save_path):
        info_dict = {'이름':name.strip(), '주민번호':enctxt.strip(), '아이디':Id.strip(), '패스워드':enc_pw.strip()}
        df = pd.DataFrame([info_dict])
        df.to_csv(save_path + '/join_info.csv', index=False, encoding='utf-8-sig')

class Login():
    def read_file(self, save_path):
        df = pd.read_csv(save_path + '/join_info.csv')

        return df

    def login_pw_enc(self, login_pw):
        sha = sha256()
        sha.update(login_pw.encode('utf-8'))
        login_enc_pw = sha.hexdigest()

        return login_enc_pw
    
    def check_info(self, df):
        check_id = df.at[0, '아이디']
        check_pw = df.at[0, '패스워드']
        
        return check_id, check_pw
        


def Join():
    keytext='samsjang'
    ivtext='1234'
    print('회원가입')
    time.sleep(1)
    name = input('이름을 입력하세요 : ')
    code = input('주민번호를 입력하세요("-"를 제외 숫자만 입력) : ')
    Id = input('사용하실 아이디를 입력하세요 : ')
    pw = input('사용하실 비밀번호를 입력하세요 : ')
    save_path = os.getcwd() # 현재 디렉토리 반환

    cipher = myDES(keytext, ivtext)
    encmsg = cipher.code_enc(code)
    enc_pw = cipher.pw_enc(pw)
    cipher.save_file(name, encmsg, Id, enc_pw, save_path)
    time.sleep(1)
    print('가입이 완료되었습니다.')

def Log_in():
    save_path = os.getcwd() # 현재 디렉토리 반환
    time.sleep(1)
    print('로그인')
    time.sleep(1)
    login_Id = input('아이디를 입력하세요 : ')
    login_pw = input('비밀번호를 입력하세요 : ')

    LogIn = Login()
    df = LogIn.read_file(save_path)
    psd = LogIn.login_pw_enc(login_pw)
    check_id, check_pw = LogIn.check_info(df)

    if login_Id == check_id and psd == check_pw:
        print('로그인 성공')
    else:
        print('로그인 실패')
    
def main():
    join_ask = input('회원이신가요?(y/n) : ')
    if join_ask == 'n' or join_ask == 'N':
        time.sleep(1)
        ask = input('회원가입을 하시겠습니까?(y/n) : ')
        if ask == 'y' or ask == 'Y':
            Join()
            time.sleep(1)
            login_ask = input('로그인 하시겠습니까?(y/n) : ')
            if login_ask == 'y' or login_ask == 'Y':
                Log_in()
            else:
                exit()
        else:
            exit()
    elif join_ask == 'y' or join_ask == 'Y':
        time.sleep(1)
        login_ask = input('로그인 하시겠습니까?(y/n) : ')
        if login_ask == 'y' or login_ask == 'Y':
            Log_in()
        else:
            exit()
        
if __name__ == '__main__':
    main()
